# Library


In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import pyexasol

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# server

In [3]:
# # insider db
# wb = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# exasol db
ws = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYS')
wb = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='vsyse')
wbt = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='wbt')

# fold path

In [4]:
drive = 'D'

In [5]:
ods_fold = f'{drive}:/data/big2/ODS'
big2_fold = f'{drive}:/data/big2'
raw_fold = f'{drive}:/data/big2/BD3/raw'
analysis_fold = f'{drive}:/data/big2/BD3/an'
df_fold = f'{drive}:/data/big2/BD3/df'

# Load

## 등록정보(STD_CEG_CAR_MIG)

In [41]:
# exasol db
# 4m 21s
# car = wb.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_MNG_NO FROM STD_CEG_CAR_MIG;")
# test 만개
car = wb.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_MNG_NO FROM STD_CEG_CAR_MIG WHERE ROWNUM <= 10000;")

In [42]:
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_MNG_NO':'차량관리번호'
}

In [43]:
carr = car.rename(columns=car_ch_col)

In [44]:
carr.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '차량관리번호'],
      dtype='object')

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [10]:
# 3.8s
src = wb.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY FROM STD_CEG_CAR_SRC_MIG;")

In [11]:
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
}

In [12]:
srcr = src.rename(columns=src_ch_col)
srcr.columns

Index(['제원관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식'], dtype='object')

## 법정동코드(STD_BJCD_INFO)

In [13]:
# 1.3s
code = wb.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46333 entries, 0 to 46332
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       46333 non-null  int64 
 1   STDG_CTPV_NM  46333 non-null  object
 2   STDG_SGG_NM   46310 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [14]:
code.columns

Index(['STDG_CD', 'STDG_CTPV_NM', 'STDG_SGG_NM'], dtype='object')

In [15]:
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구',  
}

In [16]:
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도', '시군구'], dtype='object')

## 등록이력(CEG_CAR_HISTORY_MIG)

In [17]:
# 1m 8.3s
his = ws.export_to_pandas("SELECT VHCL_ERSR_YN, CHNG_DE, VHMNO FROM CEG_CAR_HISTORY_MIG;")
his.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50044247 entries, 0 to 50044246
Data columns (total 3 columns):
 #   Column        Dtype  
---  ------        -----  
 0   VHCL_ERSR_YN  object 
 1   CHNG_DE       float64
 2   VHMNO         object 
dtypes: float64(1), object(2)
memory usage: 1.1+ GB


In [18]:
his_ch_col = {
    'VHCL_ERSR_YN':'차량말소YN', 
    'CHNG_DE':'변경일자',
    'VHMNO':'차량관리번호'
}

In [19]:
hisr = his.rename(columns=his_ch_col)
hisr.columns

Index(['차량말소YN', '변경일자', '차량관리번호'], dtype='object')

# 전처리

## 중복 차대번호 제거

In [45]:
carr.shape, len(carr['차대번호'].unique())

((10000, 11), 10000)

In [46]:
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(10000, 11)

## 배출가스등급 코드 변환

In [47]:
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)
carr['배출가스등급'].unique()

array(['5', '3', '2', '4', 'X'], dtype=object)

## 차종 코드 변환

In [48]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['승용', '화물', '승합', '특수'], dtype=object)

## 용도 코드 변환

In [49]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

## 등록정보 말소 제거

In [50]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(3812, 11)

## 연료 코드 변환

In [51]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['경유', '휘발유', 'CNG(압축천연가스)', nan, 'LPG(액화석유가스)', '기타연료',
       '휘발유 하이브리드', '전기', '알코올', 'CNG 하이브리드', '태양열', '등유', '경유 하이브리드',
       'LPG 하이브리드', '수소', 'LNG(액화천연가스)'], dtype=object)

## 차종유형 코드 변환

In [52]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '소형', '중형', '경형', nan], dtype=object)

## 등록&제원 병합

In [53]:
# 19.4s
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(3812, 17)

## 등록&제원&법정동코드 병합(df)

In [54]:
df = cs.merge(coder, on='법정동코드', how='left')

In [55]:
df.shape

(3812, 19)

### 매칭 안되는 지역 처리

In [56]:
df['시도'].isnull().sum()

0

In [57]:
df.loc[df['시도'].isnull() == True, ['법정동코드', '시도', '시군구']]

,법정동코드,시도,시군구


In [58]:
df.loc[df['시도'].isnull() == True, '법정동코드'].unique()

array([], dtype=int64)

In [59]:
# 주소 수정
df.loc[df['법정동코드'] == 4161011600, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 능평동
df.loc[df['법정동코드'] == 4161012000, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 양벌동
df.loc[df['법정동코드'] == 4161011800, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 추자동
df.loc[df['법정동코드'] == 4161011500, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 신현동
df.loc[df['법정동코드'] == 4161011900, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 매산동
df.loc[df['법정동코드'] == 4161011700, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 문형동
df.loc[df['법정동코드'] == 4161011400, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 고산동
df.loc[df['법정동코드'] == 4165052000, ['시도', '시군구']] = ['경기도', '포천시'] # 경기도 포천시 선단동
df.loc[df['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시'] # 경기도 양주시 회천3동

In [60]:
df['시도'].isnull().sum()

0

## 등록(말소 유지) & 제원 병합

In [61]:
# 10.3s
cse = carr.merge(srcr, on='제원관리번호', how='left')

In [62]:
cse.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '차량관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식'],
      dtype='object')

## 등록&제원&이력 병합

In [63]:
# 2m 6.0s
ersr = cse.merge(hisr, on='차량관리번호', how='left')

In [64]:
ersr.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN_x', '제원관리번호', '차량연식',
       '차종', '용도', '최초등록일자', '차량관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식',
       '엔진형식', '차량말소YN_y', '변경일자'],
      dtype='object')

In [65]:
# 1. 등록 차량말소와 등록이력 차량 말소 둘 모두 해당되는 데이터 추출
ersr = ersr.loc[(ersr['차량말소YN_x'] == 'Y') & (ersr['차량말소YN_y'] == 'Y')].reset_index(drop=True)

In [66]:
# 2. 변경일자 기준 최신 데이터만 남기고 차대번호 중복 제거
ersr = ersr.sort_values('변경일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)

In [67]:
# 3. 변경일자 2019.01.01 이후만 추출
ersr = ersr[ersr['변경일자'] >= 20190101].reset_index(drop=True)

## 등록&제원&이력&법정동코드 병합(errc)

In [68]:
# 매칭 후 '시도' 빈값 없음
errc = ersr.merge(coder, on='법정동코드', how='left')

# 분석

## 연료 하이브리드 수정

In [69]:
# about 23.6s
df.loc[(df['연료'] == '휘발유') | (df['연료'] == '휘발유 하이브리드'), 'fuel'] = '휘발유'
df.loc[(df['연료'] == '경유') | (df['연료'] == '경유 하이브리드'), 'fuel'] = '경유'
df.loc[(df['연료'] == 'LPG(액화석유가스)') | (df['연료'] == 'LPG 하이브리드'), 'fuel'] = 'LPG'
df.loc[(df['연료'] == 'CNG(압축천연가스)') | (df['연료'] == 'CNG 하이브리드'), 'fuel'] = 'CNG'
df.loc[(df['연료'] == 'LNG(액화천연가스)') | (df['연료'] == 'LNG 하이브리드'), 'fuel'] = 'LNG'
df.loc[df['연료'] == '전기', 'fuel'] = '전기'
df.loc[df['연료'] == '수소', 'fuel'] = '수소'
df.loc[df['연료'] == '태양열', 'fuel'] = '태양열'
df.loc[df['연료'] == '알코올', 'fuel'] = '알코올'
df.loc[df['연료'] == '등유', 'fuel'] = '등유'
df.loc[df['연료'] == '기타연료', 'fuel'] = '기타연료'
df.loc[df['연료'].isnull() == True, 'fuel'] = np.nan

In [70]:
# about 3.3s
errc.loc[(errc['연료'] == '휘발유') | (errc['연료'] == '휘발유 하이브리드'), 'fuel'] = '휘발유'
errc.loc[(errc['연료'] == '경유') | (errc['연료'] == '경유 하이브리드'), 'fuel'] = '경유'
errc.loc[(errc['연료'] == 'LPG(액화석유가스)') | (errc['연료'] == 'LPG 하이브리드'), 'fuel'] = 'LPG'
errc.loc[(errc['연료'] == 'CNG(압축천연가스)') | (errc['연료'] == 'CNG 하이브리드'), 'fuel'] = 'CNG'
errc.loc[(errc['연료'] == 'LNG(액화천연가스)') | (errc['연료'] == 'LNG 하이브리드'), 'fuel'] = 'LNG'
errc.loc[errc['연료'] == '전기', 'fuel'] = '전기'
errc.loc[errc['연료'] == '수소', 'fuel'] = '수소'
errc.loc[errc['연료'] == '태양열', 'fuel'] = '태양열'
errc.loc[errc['연료'] == '알코올', 'fuel'] = '알코올'
errc.loc[errc['연료'] == '등유', 'fuel'] = '등유'
errc.loc[errc['연료'] == '기타연료', 'fuel'] = '기타연료'
errc.loc[errc['연료'].isnull() == True, 'fuel'] = np.nan

In [71]:
# 6s
# 수도권 : 서울특별시, 인천광역시, 경기도
# 비수도권 : 수도권 외
df.loc[(df['시도'] == '서울특별시') | (df['시도'] == '인천광역시') | (df['시도'] == '경기도'), '지역'] = '수도권'
df['지역'] = df['지역'].fillna('비수도권')

In [72]:
dfm = df.copy()

In [73]:
dfm['최초등록일자'] = dfm['최초등록일자'].astype('str')
dfm['최초등록일자_년'] = dfm['최초등록일자'].str[:4]
dfm['최초등록일자_월'] = dfm['최초등록일자'].str[4:6]
dfm['최초등록일자_일'] = dfm['최초등록일자'].str[6:8]

In [74]:
# 시군구명 앞쪽 지역명만 남기기(dfm)
dfm['시군구_수정'] = dfm['시군구'].str.split(' ').str[0]

In [75]:
# 수도권 : 서울특별시, 인천광역시, 경기도
# 비수도권 : 수도권 외
errc.loc[(errc['시도'] == '서울특별시') | (errc['시도'] == '인천광역시') | (errc['시도'] == '경기도'), '지역'] = '수도권'
errc['지역'] = errc['지역'].fillna('비수도권')

In [76]:
errc['변경일자'] = errc['변경일자'].astype('str')
errc['변경일자_년'] = errc['변경일자'].str[:4]
errc['변경일자_월'] = errc['변경일자'].str[4:6]
errc['변경일자_일'] = errc['변경일자'].str[6:8]

In [77]:
# 시군구명 앞쪽 지역명만 남기기(errc)
errc['시군구_수정'] = errc['시군구'].str.split(' ').str[0]

In [78]:
# 2m 41.5s
dfm2 = dfm.loc[(dfm['fuel'] == '경유') | (dfm['fuel'] == '휘발유') | (dfm['fuel'] == 'LPG') | (dfm['fuel'] == '전기') | (dfm['fuel'] == '수소')].reset_index(drop=True)
errc2 = errc.loc[(errc['fuel'] == '경유') | (errc['fuel'] == '휘발유') | (errc['fuel'] == 'LPG') | (errc['fuel'] == '전기') | (errc['fuel'] == '수소')].reset_index(drop=True)

## 등급, 지역별 차량현황

In [79]:
# 2022년 차량 대수
grp1 = dfm2.groupby(['지역', '시도', '배출가스등급'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = '2022'
grp1 = grp1[['연도', '지역', '시도', '배출가스등급', '차량대수']]
grp1

,연도,지역,시도,배출가스등급,차량대수
0,2022,비수도권,강원도,2,1
1,2022,비수도권,강원도,3,35
2,2022,비수도권,강원도,4,12
3,2022,비수도권,강원도,5,80
4,2022,비수도권,경상남도,2,3
...,...,...,...,...,...
56,2022,수도권,서울특별시,5,134
57,2022,수도권,인천광역시,2,3
58,2022,수도권,인천광역시,3,75
59,2022,수도권,인천광역시,4,26


In [80]:
# 4년간 차량 통계 기본 데이터셋
yr_list = []
rgn_list = []
ctpv_list = []
grd_list = []
for ctpv in grp1['시도'].unique():
    if ('서울특별시' in ctpv) or ('인천광역시' in ctpv) or ('경기도' in ctpv):
        rgn = '수도권'
    else:
        rgn = '비수도권'
        for grd in ['1', '2', '3', '4', '5', 'X']:
            for yr in ['2019', '2020', '2021', '2022']:
                yr_list.append(yr)
                rgn_list.append(rgn)
                ctpv_list.append(ctpv)
                grd_list.append(grd)
base = pd.DataFrame({'연도':yr_list, '지역':rgn_list, '시도':ctpv_list, '배출가스등급':grd_list})

In [81]:
base.head()

,연도,지역,시도,배출가스등급
0,2019,비수도권,강원도,1
1,2020,비수도권,강원도,1
2,2021,비수도권,강원도,1
3,2022,비수도권,강원도,1
4,2019,비수도권,강원도,2


In [82]:
# 연도별 등록대수
grp2 = dfm2.groupby(['최초등록일자_년', '지역', '시도', '배출가스등급'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,지역,시도,배출가스등급,등록대수
0,2000,비수도권,강원도,2,1
1,2000,비수도권,강원도,3,14
2,2000,비수도권,강원도,4,10
3,2000,비수도권,강원도,5,33
4,2000,비수도권,경상남도,2,2
...,...,...,...,...,...
157,2006,비수도권,대전광역시,5,1
158,2006,비수도권,충청북도,5,1
159,2006,수도권,경기도,2,1
160,2006,수도권,경기도,5,1


In [83]:
# 연도별 말소대수
grp3 = errc2.groupby(['변경일자_년', '지역', '시도', '배출가스등급'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,지역,시도,배출가스등급,말소대수
0,2019,비수도권,강원도,2,1
1,2019,비수도권,강원도,3,21
2,2019,비수도권,강원도,4,5
3,2019,비수도권,강원도,5,24
4,2019,비수도권,경상남도,2,1
...,...,...,...,...,...
209,2022,수도권,서울특별시,4,7
210,2022,수도권,서울특별시,5,16
211,2022,수도권,인천광역시,3,8
212,2022,수도권,인천광역시,4,4


In [84]:
base1 = base.merge(grp1, on=['연도', '지역', '시도', '배출가스등급'], how='left')
base1

,연도,지역,시도,배출가스등급,차량대수
0,2019,비수도권,강원도,1,NaN
1,2020,비수도권,강원도,1,NaN
2,2021,비수도권,강원도,1,NaN
3,2022,비수도권,강원도,1,NaN
4,2019,비수도권,강원도,2,NaN
...,...,...,...,...,...
331,2022,비수도권,충청북도,5,81.00
332,2019,비수도권,충청북도,X,NaN
333,2020,비수도권,충청북도,X,NaN
334,2021,비수도권,충청북도,X,NaN


In [85]:
base2 = base1.merge(grp2, on=['연도', '지역', '시도', '배출가스등급'], how='left')
base2

,연도,지역,시도,배출가스등급,차량대수,등록대수
0,2019,비수도권,강원도,1,NaN,NaN
1,2020,비수도권,강원도,1,NaN,NaN
2,2021,비수도권,강원도,1,NaN,NaN
3,2022,비수도권,강원도,1,NaN,NaN
4,2019,비수도권,강원도,2,NaN,NaN
...,...,...,...,...,...,...
331,2022,비수도권,충청북도,5,81.00,NaN
332,2019,비수도권,충청북도,X,NaN,NaN
333,2020,비수도권,충청북도,X,NaN,NaN
334,2021,비수도권,충청북도,X,NaN,NaN


In [86]:
base3 = base2.merge(grp3, on=['연도', '지역', '시도', '배출가스등급'], how='left')
base3

,연도,지역,시도,배출가스등급,차량대수,등록대수,말소대수
0,2019,비수도권,강원도,1,NaN,NaN,NaN
1,2020,비수도권,강원도,1,NaN,NaN,NaN
2,2021,비수도권,강원도,1,NaN,NaN,NaN
3,2022,비수도권,강원도,1,NaN,NaN,NaN
4,2019,비수도권,강원도,2,NaN,NaN,1.00
...,...,...,...,...,...,...,...
331,2022,비수도권,충청북도,5,81.00,NaN,11.00
332,2019,비수도권,충청북도,X,NaN,NaN,1.00
333,2020,비수도권,충청북도,X,NaN,NaN,NaN
334,2021,비수도권,충청북도,X,NaN,NaN,NaN


In [87]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [88]:
n = 4
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [89]:
base3

,연도,지역,시도,배출가스등급,차량대수,등록대수,말소대수
0,2019,비수도권,강원도,1,0.00,0.00,0.00
1,2020,비수도권,강원도,1,0.00,0.00,0.00
2,2021,비수도권,강원도,1,0.00,0.00,0.00
3,2022,비수도권,강원도,1,0.00,0.00,0.00
4,2019,비수도권,강원도,2,1.00,0.00,1.00
...,...,...,...,...,...,...,...
331,2022,비수도권,충청북도,5,81.00,0.00,11.00
332,2019,비수도권,충청북도,X,0.00,0.00,1.00
333,2020,비수도권,충청북도,X,0.00,0.00,0.00
334,2021,비수도권,충청북도,X,0.00,0.00,0.00


In [90]:
base3.columns

Index(['연도', '지역', '시도', '배출가스등급', '차량대수', '등록대수', '말소대수'], dtype='object')

In [91]:
df1 = base3[['연도', '지역', '시도', '배출가스등급', '차량대수']]

In [92]:
cdict = {
    '연도':'YR', 
    '지역':'RGN', 
    '시도':'CTPV', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '차량대수':'VHCL_MKCNT',
}

In [93]:
STD_BD_CAR_CURSTT_MOD = df1.rename(columns=cdict)
STD_BD_CAR_CURSTT_MOD.columns

Index(['YR', 'RGN', 'CTPV', 'EXHST_GAS_GRD_CD', 'VHCL_MKCNT'], dtype='object')

In [94]:
STD_BD_CAR_CURSTT_MOD.head()

,YR,RGN,CTPV,EXHST_GAS_GRD_CD,VHCL_MKCNT
0,2019,비수도권,강원도,1,0.00
1,2020,비수도권,강원도,1,0.00
2,2021,비수도권,강원도,1,0.00
3,2022,비수도권,강원도,1,0.00
4,2019,비수도권,강원도,2,1.00


### [출력] STD_BD_CAR_CURSTT_MOD

In [83]:
expdf = STD_BD_CAR_CURSTT_MOD
table_nm = 'STD_BD_CAR_CURSTT_MOD'.upper()

In [84]:
# 테이블 생성
sql = 'create table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
wbt.execute(sql)

<ExaStatement session_id=1769751926526181376 stmt_idx=1>

In [85]:
# 데이터 추가
# 5s
expdf = STD_BD_CAR_CURSTT_MOD
table_nm = 'STD_BD_CAR_CURSTT_MOD'.upper()
wbt.import_from_pandas(expdf, table_nm)

## 등급, 연료별 차량현황

In [95]:
# 2022년 차량 대수
grp1 = dfm2.groupby(['fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = '2022'
grp1 = grp1[['연도', 'fuel', '배출가스등급', '차량대수']]

In [96]:
grp1.head()

,연도,fuel,배출가스등급,차량대수
0,2022,LPG,2,14
1,2022,LPG,3,428
2,2022,LPG,4,170
3,2022,경유,5,1660
4,2022,휘발유,2,1


In [97]:
# 4년간 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
grd_list = []
for fuel in grp1['fuel'].unique():
    for grd in ['1', '2', '3', '4', '5', 'X']:
        for yr in ['2019', '2020', '2021', '2022']:
            yr_list.append(yr)
            fuel_list.append(fuel)
            grd_list.append(grd)
base = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list})

In [98]:
base.head()

,연도,fuel,배출가스등급
0,2019,LPG,1
1,2020,LPG,1
2,2021,LPG,1
3,2022,LPG,1
4,2019,LPG,2


In [99]:
# 연도별 등록대수
grp2 = dfm2.groupby(['최초등록일자_년', 'fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,fuel,배출가스등급,등록대수
0,2000,LPG,2,7
1,2000,LPG,3,259
2,2000,LPG,4,147
3,2000,경유,5,751
4,2000,휘발유,3,297
5,2000,휘발유,4,301
6,2001,LPG,2,6
7,2001,LPG,3,90
8,2001,LPG,4,21
9,2001,경유,5,544


In [100]:
# 연도별 말소대수
grp3 = errc2.groupby(['변경일자_년', 'fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,fuel,배출가스등급,말소대수
0,2019,LPG,2,4
1,2019,LPG,3,183
2,2019,LPG,4,58
3,2019,LPG,X,1
4,2019,경유,5,621
5,2019,경유,X,2
6,2019,휘발유,2,1
7,2019,휘발유,3,493
8,2019,휘발유,4,191
9,2019,휘발유,X,3


In [101]:
base1 = base.merge(grp1, on=['연도', 'fuel', '배출가스등급'], how='left')
base1

,연도,fuel,배출가스등급,차량대수
0,2019,LPG,1,NaN
1,2020,LPG,1,NaN
2,2021,LPG,1,NaN
3,2022,LPG,1,NaN
4,2019,LPG,2,NaN
...,...,...,...,...
67,2022,휘발유,5,NaN
68,2019,휘발유,X,NaN
69,2020,휘발유,X,NaN
70,2021,휘발유,X,NaN


In [102]:
base2 = base1.merge(grp2, on=['연도', 'fuel', '배출가스등급'], how='left')
base2

,연도,fuel,배출가스등급,차량대수,등록대수
0,2019,LPG,1,NaN,NaN
1,2020,LPG,1,NaN,NaN
2,2021,LPG,1,NaN,NaN
3,2022,LPG,1,NaN,NaN
4,2019,LPG,2,NaN,NaN
...,...,...,...,...,...
67,2022,휘발유,5,NaN,NaN
68,2019,휘발유,X,NaN,NaN
69,2020,휘발유,X,NaN,NaN
70,2021,휘발유,X,NaN,NaN


In [103]:
base3 = base2.merge(grp3, on=['연도', 'fuel', '배출가스등급'], how='left')
base3

,연도,fuel,배출가스등급,차량대수,등록대수,말소대수
0,2019,LPG,1,NaN,NaN,NaN
1,2020,LPG,1,NaN,NaN,NaN
2,2021,LPG,1,NaN,NaN,NaN
3,2022,LPG,1,NaN,NaN,NaN
4,2019,LPG,2,NaN,NaN,4.00
...,...,...,...,...,...,...
67,2022,휘발유,5,NaN,NaN,NaN
68,2019,휘발유,X,NaN,NaN,3.00
69,2020,휘발유,X,NaN,NaN,NaN
70,2021,휘발유,X,NaN,NaN,NaN


In [104]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [105]:
n = 4
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [106]:
base3

,연도,fuel,배출가스등급,차량대수,등록대수,말소대수
0,2019,LPG,1,0.00,0.00,0.00
1,2020,LPG,1,0.00,0.00,0.00
2,2021,LPG,1,0.00,0.00,0.00
3,2022,LPG,1,0.00,0.00,0.00
4,2019,LPG,2,17.00,0.00,4.00
...,...,...,...,...,...,...
67,2022,휘발유,5,0.00,0.00,0.00
68,2019,휘발유,X,0.00,0.00,3.00
69,2020,휘발유,X,0.00,0.00,0.00
70,2021,휘발유,X,0.00,0.00,0.00


In [107]:
df2 = base3[['연도', 'fuel', '배출가스등급', '차량대수']]

In [108]:
cdict = {
    '연도':'YR', 
    '연료':'FUEL_CD', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '차량대수':'VHCL_MKCNT',
}

In [109]:
STD_BD_CAR_CURSTT_MOD2 = df2.rename(columns=cdict)
STD_BD_CAR_CURSTT_MOD2.columns

Index(['YR', 'fuel', 'EXHST_GAS_GRD_CD', 'VHCL_MKCNT'], dtype='object')

In [110]:
STD_BD_CAR_CURSTT_MOD2.head()

,YR,fuel,EXHST_GAS_GRD_CD,VHCL_MKCNT
0,2019,LPG,1,0.00
1,2020,LPG,1,0.00
2,2021,LPG,1,0.00
3,2022,LPG,1,0.00
4,2019,LPG,2,17.00


### [출력] STD_BD_CAR_CURSTT_MOD2

In [101]:
expdf = STD_BD_CAR_CURSTT_MOD2
table_nm = 'STD_BD_CAR_CURSTT_MOD2'.upper()

In [102]:
# 테이블 생성
sql = 'create table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
wbt.execute(sql)

<ExaStatement session_id=1769751926526181376 stmt_idx=3>

In [103]:
# 데이터 추가
# 5s
expdf = STD_BD_CAR_CURSTT_MOD2
table_nm = 'STD_BD_CAR_CURSTT_MOD2'.upper()
wbt.import_from_pandas(expdf, table_nm)

## 전체 차량 현황(등급, 차종, 지역별 차량현황)

In [111]:
df3 = dfm2[[
    '법정동코드', 
    '차종',
    '용도', 
    '차대번호', 
    '제원관리번호', 
    '배출가스인증번호', 
    '배출가스등급', 
    '차량말소YN', 
    'fuel', 
    '시도', 
    '시군구_수정'
]]

In [112]:
cdict = {
    '법정동코드':'STDG_CD', 
    '차종':'VHCTY_CD',
    '용도':'PURPS_CD2', 
    '차대번호':'VIN', 
    '제원관리번호':'MANG_MNG_NO', 
    '배출가스인증번호':'EXHST_GAS_CERT_NO', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '차량말소YN':'VHCL_ERSR_YN', 
    'fuel':'FUEL_CD', 
    '시도':'CTPV', 
    '시군구_수정':'SGG', 
}

In [113]:
STD_BD_CAR_CURSTT = df3.rename(columns=cdict)
STD_BD_CAR_CURSTT.columns

Index(['STDG_CD', 'VHCTY_CD', 'PURPS_CD2', 'VIN', 'MANG_MNG_NO',
       'EXHST_GAS_CERT_NO', 'EXHST_GAS_GRD_CD', 'VHCL_ERSR_YN', 'FUEL_CD',
       'CTPV', 'SGG'],
      dtype='object')

### [출력] STD_BD_CAR_CURSTT

In [114]:
expdf = STD_BD_CAR_CURSTT
table_nm = 'STD_BD_CAR_CURSTT'.upper()

In [115]:
# 테이블 생성
sql = 'create table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
wbt.execute(sql)

<ExaStatement session_id=1769921362201477120 stmt_idx=1>

In [129]:
# 데이터 추가
# 3m 51.1s
expdf = STD_BD_CAR_CURSTT
table_nm = 'STD_BD_CAR_CURSTT'.upper()
wbt.import_from_pandas(expdf, table_nm)

## 지역, 연료, 연도별 차량 현황 분석

In [116]:
# 2022년 차량 대수
grp1 = dfm2.groupby(['fuel', '시도'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = '2022'
grp1 = grp1[['연도', 'fuel', '시도', '차량대수']]

In [117]:
grp1.head()

,연도,fuel,시도,차량대수
0,2022,LPG,강원도,13
1,2022,LPG,경기도,141
2,2022,LPG,경상남도,38
3,2022,LPG,경상북도,44
4,2022,LPG,광주광역시,11


In [118]:
# 4년간 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
ctpv_list = []
for fuel in grp1['fuel'].unique():
    for ctpv in grp1['시도'].unique():
        for yr in ['2019', '2020', '2021', '2022']:
            yr_list.append(yr)
            fuel_list.append(fuel)
            ctpv_list.append(ctpv)
base = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '시도':ctpv_list})

In [119]:
base.head()

,연도,fuel,시도
0,2019,LPG,강원도
1,2020,LPG,강원도
2,2021,LPG,강원도
3,2022,LPG,강원도
4,2019,LPG,경기도


In [120]:
# 연도별 등록대수
grp2 = dfm2.groupby(['최초등록일자_년', 'fuel', '시도'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,fuel,시도,등록대수
0,2000,LPG,강원도,11
1,2000,LPG,경기도,99
2,2000,LPG,경상남도,26
3,2000,LPG,경상북도,24
4,2000,LPG,광주광역시,7
...,...,...,...,...
149,2006,경유,경기도,1
150,2006,경유,대전광역시,1
151,2006,경유,충청북도,1
152,2006,휘발유,강원도,3


In [121]:
# 연도별 말소대수
grp3 = errc2.groupby(['변경일자_년', 'fuel', '시도'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,fuel,시도,말소대수
0,2019,LPG,강원도,9
1,2019,LPG,경기도,46
2,2019,LPG,경상남도,20
3,2019,LPG,경상북도,13
4,2019,LPG,광주광역시,10
...,...,...,...,...
192,2022,휘발유,전라남도,6
193,2022,휘발유,전라북도,3
194,2022,휘발유,제주특별자치도,4
195,2022,휘발유,충청남도,6


In [122]:
base1 = base.merge(grp1, on=['연도', 'fuel', '시도'], how='left')
base1

,연도,fuel,시도,차량대수
0,2019,LPG,강원도,NaN
1,2020,LPG,강원도,NaN
2,2021,LPG,강원도,NaN
3,2022,LPG,강원도,13.00
4,2019,LPG,경기도,NaN
...,...,...,...,...
199,2022,휘발유,충청남도,50.00
200,2019,휘발유,충청북도,NaN
201,2020,휘발유,충청북도,NaN
202,2021,휘발유,충청북도,NaN


In [123]:
base2 = base1.merge(grp2, on=['연도', 'fuel', '시도'], how='left')
base2

,연도,fuel,시도,차량대수,등록대수
0,2019,LPG,강원도,NaN,NaN
1,2020,LPG,강원도,NaN,NaN
2,2021,LPG,강원도,NaN,NaN
3,2022,LPG,강원도,13.00,NaN
4,2019,LPG,경기도,NaN,NaN
...,...,...,...,...,...
199,2022,휘발유,충청남도,50.00,NaN
200,2019,휘발유,충청북도,NaN,NaN
201,2020,휘발유,충청북도,NaN,NaN
202,2021,휘발유,충청북도,NaN,NaN


In [124]:
base3 = base2.merge(grp3, on=['연도', 'fuel', '시도'], how='left')
base3

,연도,fuel,시도,차량대수,등록대수,말소대수
0,2019,LPG,강원도,NaN,NaN,9.00
1,2020,LPG,강원도,NaN,NaN,6.00
2,2021,LPG,강원도,NaN,NaN,7.00
3,2022,LPG,강원도,13.00,NaN,1.00
4,2019,LPG,경기도,NaN,NaN,46.00
...,...,...,...,...,...,...
199,2022,휘발유,충청남도,50.00,NaN,6.00
200,2019,휘발유,충청북도,NaN,NaN,28.00
201,2020,휘발유,충청북도,NaN,NaN,32.00
202,2021,휘발유,충청북도,NaN,NaN,22.00


In [125]:
base3[['차량대수', '등록대수', '말소대수']].isnull().sum()

차량대수    153
등록대수    204
말소대수      7
dtype: int64

In [126]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [127]:
n = 4
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [128]:
base3

,연도,fuel,시도,차량대수,등록대수,말소대수
0,2019,LPG,강원도,27.00,0.00,9.00
1,2020,LPG,강원도,21.00,0.00,6.00
2,2021,LPG,강원도,14.00,0.00,7.00
3,2022,LPG,강원도,13.00,0.00,1.00
4,2019,LPG,경기도,260.00,0.00,46.00
...,...,...,...,...,...,...
199,2022,휘발유,충청남도,50.00,0.00,6.00
200,2019,휘발유,충청북도,100.00,0.00,28.00
201,2020,휘발유,충청북도,68.00,0.00,32.00
202,2021,휘발유,충청북도,46.00,0.00,22.00


In [129]:
gp = base3.groupby(['연도', '시도', 'fuel'], as_index=False)['차량대수'].sum()

In [130]:
gp['연료비율'] = round((gp['차량대수'] / gp.groupby(['연도', '시도'])['차량대수'].transform('sum')), 2)

In [131]:
gp1 = gp[['연도', 'fuel', '시도', '차량대수', '연료비율']]

In [132]:
cdict = {
    '연도':'YR', 
    'fuel':'FUEL_CD', 
    '시도':'CTPV', 
    '차량대수':'VHCL_MKCNT', 
    '연료비율':'FUEL_RT'
}

In [133]:
STD_BD_CAR_REG_MKCNT = gp1.rename(columns=cdict)
STD_BD_CAR_REG_MKCNT.columns

Index(['YR', 'FUEL_CD', 'CTPV', 'VHCL_MKCNT', 'FUEL_RT'], dtype='object')

In [134]:
STD_BD_CAR_REG_MKCNT.head()

,YR,FUEL_CD,CTPV,VHCL_MKCNT,FUEL_RT
0,2019,LPG,강원도,27.00,0.09
1,2019,경유,강원도,185.00,0.62
2,2019,휘발유,강원도,84.00,0.28
3,2019,LPG,경기도,260.00,0.17
4,2019,경유,경기도,543.00,0.35


### [출력] STD_BD_CAR_REG_MKCNT

In [ ]:
expdf = STD_BD_CAR_REG_MKCNT
table_nm = 'STD_BD_CAR_REG_MKCNT'.upper()

In [ ]:
# 테이블 생성
sql = 'create table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
wbt.execute(sql)

<ExaStatement session_id=1769723445900345344 stmt_idx=1>

In [ ]:
# 데이터 추가
# 7s
expdf = STD_BD_CAR_REG_MKCNT
table_nm = 'STD_BD_CAR_REG_MKCNT'.upper()
wbt.import_from_pandas(expdf, table_nm)

## 내연차 연료, 연도별 차량 현황 예측
- 경유, 휘발유, LPG

In [135]:
dfm2dgl = dfm2.loc[(dfm2['fuel'] == '경유') | (dfm2['fuel'] == '휘발유') | (dfm2['fuel'] == 'LPG')].reset_index(drop=True)
dfm2dgl.shape

(3801, 25)

In [136]:
errc2dgl = errc2.loc[(errc2['fuel'] == '경유') | (errc2['fuel'] == '휘발유') | (errc2['fuel'] == 'LPG')].reset_index(drop=True)
errc2dgl.shape

(6068, 27)

In [137]:
# 2022년 차량 대수
grp1 = dfm2dgl.groupby('fuel', as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = '2022'
grp1 = grp1[['연도', 'fuel', '차량대수']]

In [138]:
grp1.head()

,연도,fuel,차량대수
0,2022,LPG,612
1,2022,경유,1660
2,2022,휘발유,1529


In [139]:
# 4년간 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
for fuel in grp1['fuel'].unique():
    for yr in ['2019', '2020', '2021', '2022']:
        yr_list.append(yr)
        fuel_list.append(fuel)
base = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list})

In [140]:
base.head()

,연도,fuel
0,2019,LPG
1,2020,LPG
2,2021,LPG
3,2022,LPG
4,2019,경유


In [141]:
# 연도별 등록대수
grp2 = dfm2dgl.groupby(['최초등록일자_년', 'fuel'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,fuel,등록대수
0,2000,LPG,413
1,2000,경유,751
2,2000,휘발유,598
3,2001,LPG,117
4,2001,경유,544
5,2001,휘발유,630
6,2002,LPG,81
7,2002,경유,354
8,2002,휘발유,297
9,2006,LPG,1


In [142]:
# 연도별 말소대수
grp3 = errc2dgl.groupby(['변경일자_년', 'fuel'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,fuel,말소대수
0,2019,LPG,246
1,2019,경유,623
2,2019,휘발유,688
3,2020,LPG,307
4,2020,경유,1055
5,2020,휘발유,956
6,2021,LPG,193
7,2021,경유,824
8,2021,휘발유,656
9,2022,LPG,48


In [143]:
base1 = base.merge(grp1, on=['연도', 'fuel'], how='left')
base1

,연도,fuel,차량대수
0,2019,LPG,NaN
1,2020,LPG,NaN
2,2021,LPG,NaN
3,2022,LPG,612.00
4,2019,경유,NaN
5,2020,경유,NaN
6,2021,경유,NaN
7,2022,경유,1660.00
8,2019,휘발유,NaN
9,2020,휘발유,NaN


In [144]:
base2 = base1.merge(grp2, on=['연도', 'fuel'], how='left')
base2

,연도,fuel,차량대수,등록대수
0,2019,LPG,NaN,NaN
1,2020,LPG,NaN,NaN
2,2021,LPG,NaN,NaN
3,2022,LPG,612.00,NaN
4,2019,경유,NaN,NaN
5,2020,경유,NaN,NaN
6,2021,경유,NaN,NaN
7,2022,경유,1660.00,NaN
8,2019,휘발유,NaN,NaN
9,2020,휘발유,NaN,NaN


In [145]:
base3 = base2.merge(grp3, on=['연도', 'fuel'], how='left')
base3

,연도,fuel,차량대수,등록대수,말소대수
0,2019,LPG,NaN,NaN,246
1,2020,LPG,NaN,NaN,307
2,2021,LPG,NaN,NaN,193
3,2022,LPG,612.00,NaN,48
4,2019,경유,NaN,NaN,623
5,2020,경유,NaN,NaN,1055
6,2021,경유,NaN,NaN,824
7,2022,경유,1660.00,NaN,251
8,2019,휘발유,NaN,NaN,688
9,2020,휘발유,NaN,NaN,956


In [146]:
base3[['차량대수', '등록대수', '말소대수']].isnull().sum()

차량대수     9
등록대수    12
말소대수     0
dtype: int64

In [147]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [148]:
n = 4
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [149]:
base3

,연도,fuel,차량대수,등록대수,말소대수
0,2019,LPG,1160.00,0.00,246
1,2020,LPG,853.00,0.00,307
2,2021,LPG,660.00,0.00,193
3,2022,LPG,612.00,0.00,48
4,2019,경유,3790.00,0.00,623
5,2020,경유,2735.00,0.00,1055
6,2021,경유,1911.00,0.00,824
7,2022,경유,1660.00,0.00,251
8,2019,휘발유,3362.00,0.00,688
9,2020,휘발유,2406.00,0.00,956


In [150]:
die = base3.loc[base3['fuel'] == '경유', ['연도', 'fuel', '차량대수']].reset_index(drop=True)
gas = base3.loc[base3['fuel'] == '휘발유', ['연도', 'fuel', '차량대수']].reset_index(drop=True)
lpg = base3.loc[base3['fuel'] == 'LPG', ['연도', 'fuel', '차량대수']].reset_index(drop=True)

In [151]:
die['연도'] = die['연도'].astype('int')
gas['연도'] = gas['연도'].astype('int')
lpg['연도'] = lpg['연도'].astype('int')

In [152]:
fit1 = np.polyfit(die['연도'], die['차량대수'], 1)
fit2 = np.polyfit(gas['연도'], gas['차량대수'], 1)
fit3 = np.polyfit(lpg['연도'], lpg['차량대수'], 1)

In [153]:
a1, b1 = fit1
a2, b2 = fit2
a3, b3 = fit3

In [154]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '경유'
for yr in range(2023, 2036):
    pred = a1 * yr + b1
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
die_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '경유_예측':pred_list})

In [155]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '휘발유'
for yr in range(2023, 2036):
    pred = a2 * yr + b2
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
gas_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '휘발유_예측':pred_list})

In [156]:
yr_list = []
fuel_list = []
pred_list = []
fuel = 'LPG'
for yr in range(2023, 2036):
    pred = a3 * yr + b3
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
lpg_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, 'LPG_예측':pred_list})

In [157]:
die_t = pd.concat([die, die_pred], ignore_index=True)
gas_t = pd.concat([gas, gas_pred], ignore_index=True)
lpg_t = pd.concat([lpg, lpg_pred], ignore_index=True)

In [158]:
die_t = die_t.rename(columns={'차량대수':'경유_대수'})
gas_t = gas_t.rename(columns={'차량대수':'휘발유_대수'})
lpg_t = lpg_t.rename(columns={'차량대수':'LPG_대수'})

In [159]:
die_t.columns

Index(['연도', 'fuel', '경유_대수', '경유_예측'], dtype='object')

In [160]:
df5 = pd.concat([die_t[['연도', '경유_대수', '경유_예측']], gas_t[['휘발유_대수', '휘발유_예측']], lpg_t[['LPG_대수', 'LPG_예측']]], axis=1)
df5.head()

,연도,경유_대수,경유_예측,휘발유_대수,휘발유_예측,LPG_대수,LPG_예측
0,2019,3790.00,NaN,3362.00,NaN,1160.00,NaN
1,2020,2735.00,NaN,2406.00,NaN,853.00,NaN
2,2021,1911.00,NaN,1750.00,NaN,660.00,NaN
3,2022,1660.00,NaN,1529.00,NaN,612.00,NaN
4,2023,NaN,720.50,NaN,723.00,NaN,362.00


In [161]:
df5.columns

Index(['연도', '경유_대수', '경유_예측', '휘발유_대수', '휘발유_예측', 'LPG_대수', 'LPG_예측'], dtype='object')

In [162]:
df5 = df5[[
    '연도', 
    '휘발유_대수', 
    '경유_대수', 
    'LPG_대수', 
    '휘발유_예측', 
    '경유_예측', 
    'LPG_예측'
    ]]

In [163]:
cdict = {
    '연도':'YR', 
    '휘발유_대수':'GSL', 
    '경유_대수':'DSL', 
    'LPG_대수':'LPG', 
    '휘발유_예측':'GSL_PRET', 
    '경유_예측':'DSL_PRET', 
    'LPG_예측':'LPG_PRET',
}

In [164]:
STD_BD_CAR_PRET = df5.rename(columns=cdict)
STD_BD_CAR_PRET.columns

Index(['YR', 'GSL', 'DSL', 'LPG', 'GSL_PRET', 'DSL_PRET', 'LPG_PRET'], dtype='object')

In [165]:
STD_BD_CAR_PRET.tail()

,YR,GSL,DSL,LPG,GSL_PRET,DSL_PRET,LPG_PRET
12,2031,NaN,NaN,NaN,-4201.00,-5050.70,-1107.60
13,2032,NaN,NaN,NaN,-4816.50,-5772.10,-1291.30
14,2033,NaN,NaN,NaN,-5432.00,-6493.50,-1475.00
15,2034,NaN,NaN,NaN,-6047.50,-7214.90,-1658.70
16,2035,NaN,NaN,NaN,-6663.00,-7936.30,-1842.40


### [출력] STD_BD_CAR_PRET

In [ ]:
expdf = STD_BD_CAR_PRET
table_nm = 'STD_BD_CAR_PRET'.upper()

In [ ]:
# 테이블 생성
sql = 'create table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
wbt.execute(sql)

<ExaStatement session_id=1769723445900345344 stmt_idx=1>

In [ ]:
# 데이터 추가
# 7s
expdf = STD_BD_CAR_PRET
table_nm = 'STD_BD_CAR_PRET'.upper()
wbt.import_from_pandas(expdf, table_nm)

## 내연차 연료, 등급, 연도별 차량 현황 예측
- 경유, 휘발유, LPG

In [166]:
# 2022년 차량 대수
grp1 = dfm2dgl.groupby(['fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = '2022'
grp1 = grp1[['연도', 'fuel', '배출가스등급', '차량대수']]

In [167]:
grp1.head()

,연도,fuel,배출가스등급,차량대수
0,2022,LPG,2,14
1,2022,LPG,3,428
2,2022,LPG,4,170
3,2022,경유,5,1660
4,2022,휘발유,2,1


In [168]:
# 4년간 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
grd_list = []
for fuel in grp1['fuel'].unique():
    for grd in grp1['배출가스등급'].unique():
        for yr in ['2019', '2020', '2021', '2022']:
            yr_list.append(yr)
            fuel_list.append(fuel)
            grd_list.append(grd)
base = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list})

In [169]:
base.head()

,연도,fuel,배출가스등급
0,2019,LPG,2
1,2020,LPG,2
2,2021,LPG,2
3,2022,LPG,2
4,2019,LPG,3


In [170]:
# 연도별 등록대수
grp2 = dfm2dgl.groupby(['최초등록일자_년', 'fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,fuel,배출가스등급,등록대수
0,2000,LPG,2,7
1,2000,LPG,3,259
2,2000,LPG,4,147
3,2000,경유,5,751
4,2000,휘발유,3,297
5,2000,휘발유,4,301
6,2001,LPG,2,6
7,2001,LPG,3,90
8,2001,LPG,4,21
9,2001,경유,5,544


In [171]:
# 연도별 말소대수
grp3 = errc2dgl.groupby(['변경일자_년', 'fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,fuel,배출가스등급,말소대수
0,2019,LPG,2,4
1,2019,LPG,3,183
2,2019,LPG,4,58
3,2019,LPG,X,1
4,2019,경유,5,621
5,2019,경유,X,2
6,2019,휘발유,2,1
7,2019,휘발유,3,493
8,2019,휘발유,4,191
9,2019,휘발유,X,3


In [172]:
base1 = base.merge(grp1, on=['연도', 'fuel', '배출가스등급'], how='left')
base1

,연도,fuel,배출가스등급,차량대수
0,2019,LPG,2,NaN
1,2020,LPG,2,NaN
2,2021,LPG,2,NaN
3,2022,LPG,2,14.00
4,2019,LPG,3,NaN
5,2020,LPG,3,NaN
6,2021,LPG,3,NaN
7,2022,LPG,3,428.00
8,2019,LPG,4,NaN
9,2020,LPG,4,NaN


In [173]:
base2 = base1.merge(grp2, on=['연도', 'fuel', '배출가스등급'], how='left')
base2

,연도,fuel,배출가스등급,차량대수,등록대수
0,2019,LPG,2,NaN,NaN
1,2020,LPG,2,NaN,NaN
2,2021,LPG,2,NaN,NaN
3,2022,LPG,2,14.00,NaN
4,2019,LPG,3,NaN,NaN
5,2020,LPG,3,NaN,NaN
6,2021,LPG,3,NaN,NaN
7,2022,LPG,3,428.00,NaN
8,2019,LPG,4,NaN,NaN
9,2020,LPG,4,NaN,NaN


In [174]:
base3 = base2.merge(grp3, on=['연도', 'fuel', '배출가스등급'], how='left')
base3

,연도,fuel,배출가스등급,차량대수,등록대수,말소대수
0,2019,LPG,2,NaN,NaN,4.00
1,2020,LPG,2,NaN,NaN,1.00
2,2021,LPG,2,NaN,NaN,2.00
3,2022,LPG,2,14.00,NaN,NaN
4,2019,LPG,3,NaN,NaN,183.00
5,2020,LPG,3,NaN,NaN,233.00
6,2021,LPG,3,NaN,NaN,143.00
7,2022,LPG,3,428.00,NaN,35.00
8,2019,LPG,4,NaN,NaN,58.00
9,2020,LPG,4,NaN,NaN,73.00


In [175]:
base3[['차량대수', '등록대수', '말소대수']].isnull().sum()

차량대수    41
등록대수    48
말소대수    24
dtype: int64

In [176]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [177]:
n = 4
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [178]:
base3

,연도,fuel,배출가스등급,차량대수,등록대수,말소대수
0,2019,LPG,2,17.00,0.00,4.00
1,2020,LPG,2,16.00,0.00,1.00
2,2021,LPG,2,14.00,0.00,2.00
3,2022,LPG,2,14.00,0.00,0.00
4,2019,LPG,3,839.00,0.00,183.00
5,2020,LPG,3,606.00,0.00,233.00
6,2021,LPG,3,463.00,0.00,143.00
7,2022,LPG,3,428.00,0.00,35.00
8,2019,LPG,4,304.00,0.00,58.00
9,2020,LPG,4,231.00,0.00,73.00


In [179]:
die = base3.loc[base3['fuel'] == '경유', ['연도', 'fuel', '배출가스등급', '차량대수']].reset_index(drop=True)
gas = base3.loc[base3['fuel'] == '휘발유', ['연도', 'fuel', '배출가스등급', '차량대수']].reset_index(drop=True)
lpg = base3.loc[base3['fuel'] == 'LPG', ['연도', 'fuel', '배출가스등급', '차량대수']].reset_index(drop=True)

In [180]:
die['연도'] = die['연도'].astype('int')
gas['연도'] = gas['연도'].astype('int')
lpg['연도'] = lpg['연도'].astype('int')

In [181]:
die['배출가스등급'].unique()

array(['2', '3', '4', '5'], dtype=object)

In [182]:
die1 = die.loc[die['배출가스등급'] == '1'].reset_index(drop=True)
die2 = die.loc[die['배출가스등급'] == '2'].reset_index(drop=True)
die3 = die.loc[die['배출가스등급'] == '3'].reset_index(drop=True)
die4 = die.loc[die['배출가스등급'] == '4'].reset_index(drop=True)
die5 = die.loc[die['배출가스등급'] == '5'].reset_index(drop=True)

In [183]:
gas1 = gas.loc[gas['배출가스등급'] == '1'].reset_index(drop=True)
gas2 = gas.loc[gas['배출가스등급'] == '2'].reset_index(drop=True)
gas3 = gas.loc[gas['배출가스등급'] == '3'].reset_index(drop=True)
gas4 = gas.loc[gas['배출가스등급'] == '4'].reset_index(drop=True)
gas5 = gas.loc[gas['배출가스등급'] == '5'].reset_index(drop=True)

In [184]:
lpg1 = lpg.loc[lpg['배출가스등급'] == '1'].reset_index(drop=True)
lpg2 = lpg.loc[lpg['배출가스등급'] == '2'].reset_index(drop=True)
lpg3 = lpg.loc[lpg['배출가스등급'] == '3'].reset_index(drop=True)
lpg4 = lpg.loc[lpg['배출가스등급'] == '4'].reset_index(drop=True)
lpg5 = lpg.loc[lpg['배출가스등급'] == '5'].reset_index(drop=True)

In [190]:
# fit_d1 = np.polyfit(die1['연도'], die1['차량대수'], 1)
fit_d2 = np.polyfit(die2['연도'], die2['차량대수'], 1)
fit_d3 = np.polyfit(die3['연도'], die3['차량대수'], 1)
fit_d4 = np.polyfit(die4['연도'], die4['차량대수'], 1)
fit_d5 = np.polyfit(die5['연도'], die5['차량대수'], 1)

In [191]:
# fit_g1 = np.polyfit(gas1['연도'], gas1['차량대수'], 1)
fit_g2 = np.polyfit(gas2['연도'], gas2['차량대수'], 1)
fit_g3 = np.polyfit(gas3['연도'], gas3['차량대수'], 1)
fit_g4 = np.polyfit(gas4['연도'], gas4['차량대수'], 1)
fit_g5 = np.polyfit(gas5['연도'], gas5['차량대수'], 1)

In [192]:
# fit_l1 = np.polyfit(lpg1['연도'], lpg1['차량대수'], 1)
fit_l2 = np.polyfit(lpg2['연도'], lpg2['차량대수'], 1)
fit_l3 = np.polyfit(lpg3['연도'], lpg3['차량대수'], 1)
fit_l4 = np.polyfit(lpg4['연도'], lpg4['차량대수'], 1)
fit_l5 = np.polyfit(lpg5['연도'], lpg5['차량대수'], 1)

In [193]:
# ad1, bd1 = fit_d1
ad2, bd2 = fit_d2
ad3, bd3 = fit_d3
ad4, bd4 = fit_d4
ad5, bd5 = fit_d5

In [194]:
# ag1, bg1 = fit_g1
ag2, bg2 = fit_g2
ag3, bg3 = fit_g3
ag4, bg4 = fit_g4
ag5, bg5 = fit_g5

In [195]:
# al1, bl1 = fit_l1
al2, bl2 = fit_l2
al3, bl3 = fit_l3
al4, bl4 = fit_l4
al5, bl5 = fit_l5

In [196]:
# # 경유 1등급 예측
# yr_list = []
# fuel_list = []
# grd_list = []
# pred_list = []
# fuel = '경유'
# grd = '1'
# for yr in range(2023, 2036):
#     pred = ad1 * yr + bd1
#     yr_list.append(yr)
#     fuel_list.append(fuel)
#     grd_list.append(grd)
#     pred_list.append(pred)
# die1_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})
# 경유 2등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '경유'
grd = '2'
for yr in range(2023, 2036):
    pred = ad2 * yr + bd2
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
die2_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})
# 경유 3등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '경유'
grd = '3'
for yr in range(2023, 2036):
    pred = ad3 * yr + bd3
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
die3_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})
# 경유 4등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '경유'
grd = '4'
for yr in range(2023, 2036):
    pred = ad4 * yr + bd4
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
die4_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})
# 경유 5등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '경유'
grd = '5'
for yr in range(2023, 2036):
    pred = ad5 * yr + bd5
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
die5_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})

In [197]:
# # 휘발유 1등급 예측
# yr_list = []
# fuel_list = []
# grd_list = []
# pred_list = []
# fuel = '휘발유'
# grd = '1'
# for yr in range(2023, 2036):
#     pred = ag1 * yr + bg1
#     yr_list.append(yr)
#     fuel_list.append(fuel)
#     grd_list.append(grd)
#     pred_list.append(pred)
# gas1_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})
# 휘발유 2등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '휘발유'
grd = '2'
for yr in range(2023, 2036):
    pred = ag2 * yr + bg2
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
gas2_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})
# 휘발유 3등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '휘발유'
grd = '3'
for yr in range(2023, 2036):
    pred = ag3 * yr + bg3
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
gas3_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})
# 휘발유 4등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '휘발유'
grd = '4'
for yr in range(2023, 2036):
    pred = ag4 * yr + bg4
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
gas4_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})
# 휘발유 5등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '휘발유'
grd = '5'
for yr in range(2023, 2036):
    pred = ag5 * yr + bg5
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
gas5_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})

In [198]:
# # LPG 1등급 예측
# yr_list = []
# fuel_list = []
# grd_list = []
# pred_list = []
# fuel = 'LPG'
# grd = '1'
# for yr in range(2023, 2036):
#     pred = al1 * yr + bl1
#     yr_list.append(yr)
#     fuel_list.append(fuel)
#     grd_list.append(grd)
#     pred_list.append(pred)
# lpg1_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})
# LPG 2등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = 'LPG'
grd = '2'
for yr in range(2023, 2036):
    pred = al2 * yr + bl2
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
lpg2_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})
# LPG 3등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = 'LPG'
grd = '3'
for yr in range(2023, 2036):
    pred = al3 * yr + bl3
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
lpg3_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})
# LPG 4등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = 'LPG'
grd = '4'
for yr in range(2023, 2036):
    pred = al4 * yr + bl4
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
lpg4_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})
# LPG 5등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = 'LPG'
grd = '5'
for yr in range(2023, 2036):
    pred = al5 * yr + bl5
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
lpg5_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})

In [199]:
die1.head()

,연도,fuel,배출가스등급,차량대수


In [200]:
die_t = pd.concat([die1,  die2, die2_pred, die3, die3_pred, die4, die4_pred, die5, die5_pred], ignore_index=True)
gas_t = pd.concat([gas1,  gas2, gas2_pred, gas3, gas3_pred, gas4, gas4_pred, gas5, gas5_pred], ignore_index=True)
lpg_t = pd.concat([lpg1,  lpg2, lpg2_pred, lpg3, lpg3_pred, lpg4, lpg4_pred, lpg5, lpg5_pred], ignore_index=True)

In [201]:
die_t = die_t.rename(columns={'경유_예측':'차량예측'})
gas_t = gas_t.rename(columns={'휘발유_예측':'차량예측'})
lpg_t = lpg_t.rename(columns={'LPG_예측':'차량예측'})

In [202]:
df6 = pd.concat([die_t, gas_t, lpg_t], ignore_index=True)

In [203]:
df6.loc[df6['차량예측'] < 0, '차량예측'] = 0

In [204]:
df6.columns

Index(['연도', 'fuel', '배출가스등급', '차량대수', '차량예측'], dtype='object')

In [205]:
cdict = {
    '연도':'YR', 
    'fuel':'FUEL_CD', 
    '배출가스등급':'GRD', 
    '차량대수':'VHCL_MKCNT', 
    '차량예측':'VHCL_PRET', 
}

In [206]:
STD_BD_FUEL_GRD_VHCL_CURSTT_PRET = df6.rename(columns=cdict)
STD_BD_FUEL_GRD_VHCL_CURSTT_PRET.columns

Index(['YR', 'FUEL_CD', 'GRD', 'VHCL_MKCNT', 'VHCL_PRET'], dtype='object')

In [207]:
STD_BD_FUEL_GRD_VHCL_CURSTT_PRET.head()

,YR,FUEL_CD,GRD,VHCL_MKCNT,VHCL_PRET
0,2019,경유,2,0.00,NaN
1,2020,경유,2,0.00,NaN
2,2021,경유,2,0.00,NaN
3,2022,경유,2,0.00,NaN
4,2023,경유,2,NaN,0.00


### [출력] STD_BD_FUEL_GRD_VHCL_CURSTT_PRET

In [ ]:
expdf = STD_BD_FUEL_GRD_VHCL_CURSTT_PRET
table_nm = 'STD_BD_FUEL_GRD_VHCL_CURSTT_PRET'.upper()

In [ ]:
# 테이블 생성
sql = 'create table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
wbt.execute(sql)

<ExaStatement session_id=1769723445900345344 stmt_idx=1>

In [ ]:
# 데이터 추가
# 7s
expdf = STD_BD_FUEL_GRD_VHCL_CURSTT_PRET
table_nm = 'STD_BD_FUEL_GRD_VHCL_CURSTT_PRET'.upper()
wbt.import_from_pandas(expdf, table_nm)

## 무공해차 연료, 연도별 차량 현황 예측
- 전기, 수소

In [208]:
dfm2bh = dfm2.loc[(dfm2['fuel'] == '전기') | (dfm2['fuel'] == '수소')].reset_index(drop=True)
dfm2bh.shape

(0, 25)

In [209]:
errc2bh = errc2.loc[(errc2['fuel'] == '전기') | (errc2['fuel'] == '수소')].reset_index(drop=True)
errc2bh.shape

(0, 27)

In [420]:
# 2022년 차량 대수
grp1 = dfm2bh.groupby('fuel', as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = '2022'
grp1 = grp1[['연도', 'fuel', '차량대수']]

In [421]:
grp1.head()

,연도,fuel,차량대수
0,2022,수소,29489
1,2022,전기,407852


In [422]:
# 4년간 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
for fuel in grp1['fuel'].unique():
    for yr in ['2019', '2020', '2021', '2022']:
        yr_list.append(yr)
        fuel_list.append(fuel)
base = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list})

In [423]:
base.head()

,연도,fuel
0,2019,수소
1,2020,수소
2,2021,수소
3,2022,수소
4,2019,전기


In [424]:
# 연도별 등록대수
grp2 = dfm2bh.groupby(['최초등록일자_년', 'fuel'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,fuel,등록대수
0,2010,전기,10
1,2011,전기,106
2,2012,전기,369
3,2013,전기,489
4,2014,수소,5
5,2014,전기,1175
6,2015,수소,33
7,2015,전기,2767
8,2016,수소,45
9,2016,전기,3994


In [425]:
# 연도별 말소대수
grp3 = errc2bh.groupby(['변경일자_년', 'fuel'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,fuel,말소대수
0,2019,수소,5
1,2019,전기,593
2,2020,수소,19
3,2020,전기,1680
4,2021,수소,38
5,2021,전기,3950
6,2022,수소,37
7,2022,전기,1790


In [426]:
base1 = base.merge(grp1, on=['연도', 'fuel'], how='left')
base1

,연도,fuel,차량대수
0,2019,수소,NaN
1,2020,수소,NaN
2,2021,수소,NaN
3,2022,수소,29489.00
4,2019,전기,NaN
5,2020,전기,NaN
6,2021,전기,NaN
7,2022,전기,407852.00


In [427]:
base2 = base1.merge(grp2, on=['연도', 'fuel'], how='left')
base2

,연도,fuel,차량대수,등록대수
0,2019,수소,NaN,4163
1,2020,수소,NaN,5812
2,2021,수소,NaN,8516
3,2022,수소,29489.00,8136
4,2019,전기,NaN,33315
5,2020,전기,NaN,46263
6,2021,전기,NaN,100262
7,2022,전기,407852.00,128997


In [428]:
base3 = base2.merge(grp3, on=['연도', 'fuel'], how='left')
base3

,연도,fuel,차량대수,등록대수,말소대수
0,2019,수소,NaN,4163,5
1,2020,수소,NaN,5812,19
2,2021,수소,NaN,8516,38
3,2022,수소,29489.00,8136,37
4,2019,전기,NaN,33315,593
5,2020,전기,NaN,46263,1680
6,2021,전기,NaN,100262,3950
7,2022,전기,407852.00,128997,1790


In [429]:
base3[['차량대수', '등록대수', '말소대수']].isnull().sum()

차량대수    6
등록대수    0
말소대수    0
dtype: int64

In [430]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [431]:
n = 4
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [432]:
base3

,연도,fuel,차량대수,등록대수,말소대수
0,2019,수소,7119.00,4163,5
1,2020,수소,12912.00,5812,19
2,2021,수소,21390.00,8516,38
3,2022,수소,29489.00,8136,37
4,2019,전기,139750.00,33315,593
5,2020,전기,184333.00,46263,1680
6,2021,전기,280645.00,100262,3950
7,2022,전기,407852.00,128997,1790


In [433]:
bt = base3.loc[base3['fuel'] == '전기', ['연도', 'fuel', '차량대수']].reset_index(drop=True)
hy = base3.loc[base3['fuel'] == '수소', ['연도', 'fuel', '차량대수']].reset_index(drop=True)

In [434]:
bt['연도'] = bt['연도'].astype('int')
hy['연도'] = hy['연도'].astype('int')

In [435]:
fit1 = np.polyfit(bt['연도'], bt['차량대수'], 1)
fit2 = np.polyfit(hy['연도'], hy['차량대수'], 1)

In [436]:
a1, b1 = fit1
a2, b2 = fit2

In [437]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '전기'
for yr in range(2023, 2036):
    pred = a1 * yr + b1
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
bt_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '전기_예측':pred_list})

In [438]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '수소'
for yr in range(2023, 2036):
    pred = a2 * yr + b2
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
hy_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '수소_예측':pred_list})

In [439]:
bt_t = pd.concat([bt, bt_pred], ignore_index=True)
hy_t = pd.concat([hy, hy_pred], ignore_index=True)

In [440]:
bt_t = bt_t.rename(columns={'차량대수':'전기_대수'})
hy_t = hy_t.rename(columns={'차량대수':'수소_대수'})

In [441]:
hy_t.columns

Index(['연도', 'fuel', '수소_대수', '수소_예측'], dtype='object')

In [442]:
df7 = pd.concat([ bt_t[['연도', '전기_대수', '전기_예측']], hy_t[['수소_대수', '수소_예측']] ], axis=1)
df7.head()

,연도,전기_대수,전기_예측,수소_대수,수소_예측
0,2019,139750.00,NaN,7119.00,NaN
1,2020,184333.00,NaN,12912.00,NaN
2,2021,280645.00,NaN,21390.00,NaN
3,2022,407852.00,NaN,29489.00,NaN
4,2023,NaN,478299.50,NaN,36624.50


In [443]:
df7.columns

Index(['연도', '전기_대수', '전기_예측', '수소_대수', '수소_예측'], dtype='object')

In [444]:
df7 = df7[[
    '연도', 
    '전기_대수', 
    '수소_대수', 
    '전기_예측',
    '수소_예측', 
    ]]

In [445]:
cdict = {
    '연도':'YR', 
    '전기_대수':'BTYCR', 
    '수소_대수':'HY', 
    '전기_예측':'BTYCR_PRET',
    '수소_예측':'HY_PRET',
}

In [446]:
STD_BD_ECO_CAR_PRET = df7.rename(columns=cdict)
STD_BD_ECO_CAR_PRET.columns

Index(['YR', 'BTYCR', 'HY', 'BTYCR_PRET', 'HY_PRET'], dtype='object')

### [출력] STD_BD_ECO_CAR_PRET

In [ ]:
expdf = STD_BD_ECO_CAR_PRET
table_nm = 'STD_BD_ECO_CAR_PRET'.upper()

In [ ]:
# 테이블 생성
sql = 'create table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
wbt.execute(sql)

<ExaStatement session_id=1769723445900345344 stmt_idx=1>

In [ ]:
# 데이터 추가
# 7s
expdf = STD_BD_ECO_CAR_PRET
table_nm = 'STD_BD_ECO_CAR_PRET'.upper()
wbt.import_from_pandas(expdf, table_nm)

## 지역, 등급별 말소 차량 현황

In [447]:
errc2.shape

(3606874, 27)

In [448]:
df8 = errc2[[
    '법정동코드',
    '차대번호', 
    '변경일자', 
    '배출가스등급', 
    '연료'
    ]]
df8.columns

Index(['법정동코드', '차대번호', '변경일자', '배출가스등급', '연료'], dtype='object')

In [449]:
df8['연료'].unique()

array(['경유', '휘발유', 'LPG(액화석유가스)', '휘발유 하이브리드', '전기', 'LPG 하이브리드', '수소',
       '경유 하이브리드'], dtype=object)

In [450]:
chc_col = {
    '법정동코드':'STDG_CD',
    '차대번호':'VIN', 
    '변경일자':'CHG_YMD', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '연료':'FUEL_CD', 
}

In [451]:
STD_BD_GRD4_ERSR_RSLT = df8.rename(columns=chc_col)
STD_BD_GRD4_ERSR_RSLT.columns

Index(['BSPL_STDG_CD', 'VIN', 'CHG_YMD', 'EXHST_GAS_GRD_CD', 'FUEL_CD'], dtype='object')

### [출력] STD_BD_GRD4_ERSR_RSLT

In [ ]:
expdf = STD_BD_GRD4_ERSR_RSLT
table_nm = 'STD_BD_GRD4_ERSR_RSLT'.upper()

In [ ]:
# 테이블 생성
sql = 'create table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
wbt.execute(sql)

<ExaStatement session_id=1769723445900345344 stmt_idx=1>

In [ ]:
# 데이터 추가
# 7s
expdf = STD_BD_GRD4_ERSR_RSLT
table_nm = 'STD_BD_GRD4_ERSR_RSLT'.upper()
wbt.import_from_pandas(expdf, table_nm)

# code end